<a href="https://colab.research.google.com/github/HCCREN/113_10791_4_data_analysis/blob/main/113_10791_4_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TNC SCOPE READER

In [1]:
!pip install io

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

In [3]:
file_name = '10791_4_20230206_01.txt'
with open(file_name, 'r') as file:
    lines = file.readlines()

# 利用列表推導式提取曲線名稱
curve_names = [line.split('=')[1].strip() for line in lines if line.startswith('%NAME=')]
# 找到 "%[DATA]" 的索引，然後捕獲之後的數據行
try:
    data_start_index = lines.index(next(line for line in lines if '%[DATA]' in line)) + 1
except StopIteration:
    data_start_index = 0  # 如果找不到 "%[DATA]"，則從文件開始處理

data_lines = [line.strip() for line in lines[data_start_index:] if not line.startswith('%')]

# 確保曲線名稱列表長度為 10，加上 'Time' 列
dynamic_column_names = ['Time'] + curve_names[:10]

# 處理數據行，移除尾部的分號
trimmed_data_lines = [line.rstrip(';') for line in data_lines]

# 轉換為 DataFrame
data_io_trimmed = StringIO('\n'.join(trimmed_data_lines))
df = pd.read_csv(data_io_trimmed, sep=";", names=dynamic_column_names)
df['Time'] = df['Time']/100


In [15]:
df

,Time,v actual/X,v nominal/X,v actual/Y,v nominal/Y,v actual/Z,v nominal/Z,v actual/C,v nominal/C,v actual/A,v nominal/A
0,-622.05,0.000000,0.0,0.390625,0.0,-0.390625,0.0,0.000000,0.0,0.000000,0.0
1,-592.05,-0.390625,0.0,0.000000,0.0,0.390625,0.0,0.000000,0.0,0.000000,0.0
2,-562.05,0.000000,0.0,0.000000,0.0,-0.390625,0.0,0.000000,0.0,0.000000,0.0
3,-532.05,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,-502.05,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20731,621307.95,-0.781250,0.0,-0.781250,0.0,0.000000,0.0,0.000000,0.0,0.429153,0.0
20732,621337.95,0.390625,0.0,-0.390625,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
20733,621367.95,0.390625,0.0,0.000000,0.0,0.390625,0.0,0.000000,0.0,-0.429153,0.0
20734,621397.95,0.000000,0.0,0.000000,0.0,-0.390625,0.0,0.000000,0.0,0.000000,0.0


In [21]:
import plotly.graph_objects as go

# Assuming df_trimmed is the DataFrame with the toolpath data
# Extracting x, y, z columns based on the user's specification
x = df['v actual/X']
y = df['v actual/Y']
z = df['v actual/Z']

# Creating the 3D plot
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='lines+markers',  # Line with dots
                                   line=dict(color='blue', width=3),  # Blue line, width of 6
                                   marker=dict(size=3, color='blue'),
                                   name='5 axis tool path'  # Label name
                                   )])

# Updating the layout of the figure
fig.update_layout(title={'text': "5 Axis Tool Path", 'x': 0.5, 'xanchor': 'center'},
                  scene=dict(xaxis_title='x',  # X-axis label
                             yaxis_title='y',  # Y-axis label
                             zaxis_title='z'),  # Z-axis label
                  width=800,  # Width of the figure
                  height=600)  # Height of the figure

# First point trace
fig.add_trace(go.Scatter3d(x=[x.iloc[0]], y=[y.iloc[0]], z=[z.iloc[0]],
                           mode='markers',
                           marker=dict(size=10, color='red'),  # Red marker for the first point
                           name='Start'))

# Last point trace
fig.add_trace(go.Scatter3d(x=[x.iloc[-1]], y=[y.iloc[-1]], z=[z.iloc[-1]],
                           mode='markers',
                           marker=dict(size=5, color='green'),  # Green marker for the last point
                           name='End'))

# Showing the figure with adjustments
fig.show()
